In [1]:
## Paquetes
import pandas as pd
import xarray as xr
import rioxarray
from pyproj import CRS
from rasterio.enums import Resampling
from scipy.ndimage import uniform_filter, distance_transform_edt, generic_filter
import numpy as np
from xrspatial import slope, aspect

In [2]:
## Funciones
def start_of_epi_year(Y: int) -> pd.Timestamp:
    dec29 = pd.Timestamp(Y - 1, 12, 29)
    jan4  = pd.Timestamp(Y, 1, 4)
    w = dec29.weekday()
    offset = (6 - w) % 7
    first_sunday = dec29 + pd.Timedelta(days=offset)
    if first_sunday > jan4:
        first_sunday -= pd.Timedelta(days=7)
    return first_sunday

def get_epi_year_week(date: pd.Timestamp) -> tuple[int, int]:
    date = pd.Timestamp(date)
    offset = (date.weekday() + 1) % 7
    sunday = date - pd.Timedelta(days=offset)
    next_start = start_of_epi_year(sunday.year + 1)
    epi_year = sunday.year if sunday < next_start else sunday.year + 1
    start_year = start_of_epi_year(epi_year)
    epi_week = ((sunday - start_year).days // 7) + 1
    return epi_year, epi_week


def downscale_tmax_with_dem_from_epiweeks(tmax_da, dem_ds, lapse_rate=-6.5):
    """
    Downscaling de tmax por lapse rate usando DEM de alta resolución.
    
    Parámetros:
    - tmax_da: xarray.DataArray con dims ('epi_week', 'y', 'x')
    - dem_ds: xarray.Dataset con variable 'band_data' (DEM de alta resolución)
    - lapse_rate: gradiente térmico en °C/km. Por defecto: -6.5
    
    Retorna:
    - xarray.DataArray de tmax ajustado a resolución fina del DEM
    """

    # 1. Extraer DEM como DataArray
    dem_fine = dem_ds['band_data']

    # 2. Crear DEM de baja resolución con mismo tamaño espacial que tmax
    # Se ajusta usando coarsen si las resoluciones son múltiples
    factor_y = dem_fine.sizes['y'] // tmax_da.sizes['y']
    factor_x = dem_fine.sizes['x'] // tmax_da.sizes['x']
    
    dem_coarse = dem_fine.coarsen(y=factor_y, x=factor_x, boundary="trim").mean()

    # 3. Reinterpolar tmax a la resolución del DEM
    # Suponemos que 'x' e 'y' tienen coordenadas compatibles (ajustar si no)
    tmax_interp = tmax_da.interp(
        x=dem_fine['x'],
        y=dem_fine['y'],
        method='linear'
    )

    # 4. Reinterpolar DEM de baja resolución al tamaño de DEM fino
    dem_coarse_interp = dem_coarse.interp(
        x=dem_fine['x'],
        y=dem_fine['y'],
        method='linear'
    )

    # 5. Calcular diferencia de elevación
    delta_z = dem_fine - dem_coarse_interp

    # 6. Aplicar lapse rate (°C/km → °C/m)
    lapse_rate_per_m = lapse_rate / 1000
    correction = delta_z * lapse_rate_per_m

    # 7. Aplicar corrección a cada epi_week
    corrected = tmax_interp + correction

    return corrected


def calcular_tpi_mascarado(dem_da, size=3):
    elev = dem_da.values.astype(float)
    valid_mask = ~np.isnan(elev)

    elev_filled = np.where(valid_mask, elev, 0)

    sum_neighbors = uniform_filter(elev_filled, size=size, mode='constant', cval=0)
    count_neighbors = uniform_filter(valid_mask.astype(float), size=size, mode='constant', cval=0)

    mean_neighbors = np.where(count_neighbors > 0, sum_neighbors / count_neighbors, np.nan)
    tpi_array = np.where(valid_mask, elev - mean_neighbors, np.nan)

    tpi_da = xr.DataArray(tpi_array, coords=dem_da.coords, dims=dem_da.dims, name='tpi')

    # Aplicar máscara explícita
    return tpi_da.where(valid_mask)


def local_rugosity(arr):
    valid = arr[~np.isnan(arr)]
    return np.nanmax(valid) - np.nanmin(valid) if len(valid) > 0 else np.nan


## Información estaciones

In [3]:
# Ruta al archivo cargado
file_path = "D:/OneDrive - CGIAR/Desktop/downscaling/data/stations/climaticas_post_calidad.csv"

# Leer el archivo
df = pd.read_csv(file_path)

df

,Unnamed: 0,Estacion,Fecha,Precipitacion,Tmin,Tmax,Humedad relativa,Velocidad del viento,Radiación solar,Latitud,Longitud
0,1,AGUACATAL-MONTEBELLO,2014-06-01,NaN,NaN,NaN,NaN,NaN,NaN,3.487402,-76.550582
1,2,AGUACATAL-MONTEBELLO,2014-06-02,NaN,NaN,NaN,NaN,NaN,NaN,3.487402,-76.550582
2,3,AGUACATAL-MONTEBELLO,2014-06-03,NaN,NaN,NaN,NaN,NaN,NaN,3.487402,-76.550582
3,4,AGUACATAL-MONTEBELLO,2014-06-04,NaN,NaN,NaN,NaN,NaN,NaN,3.487402,-76.550582
4,5,AGUACATAL-MONTEBELLO,2014-06-05,NaN,NaN,NaN,NaN,NaN,NaN,3.487402,-76.550582
...,...,...,...,...,...,...,...,...,...,...,...
120653,120654,VILLANUEVA,2025-01-27,NaN,NaN,NaN,NaN,NaN,NaN,3.432142,-76.518319
120654,120655,VILLANUEVA,2025-01-28,NaN,NaN,NaN,NaN,NaN,NaN,3.432142,-76.518319
120655,120656,VILLANUEVA,2025-01-29,NaN,NaN,NaN,NaN,NaN,NaN,3.432142,-76.518319
120656,120657,VILLANUEVA,2025-01-30,NaN,NaN,NaN,NaN,NaN,NaN,3.432142,-76.518319


In [14]:

# Convertir a datetime y calcular semana epidemiológica
df['Fecha'] = pd.to_datetime(df['Fecha'])
# Forzar conversión de Tmax a numérico (y limpiar errores)
df['Tmin'] = pd.to_numeric(df['Tmin'], errors='coerce')
df[['epi_year', 'epi_week']] = df['Fecha'].apply(lambda d: pd.Series(get_epi_year_week(d)))

# Columna booleana: indicador de días válidos de Tmax
df['Tmin_valida'] = df['Tmin'].notna()

# Agrupar por estación, semana, y contar días válidos
agg_tmin= (
    df.groupby(['Estacion', 'epi_year', 'epi_week'], as_index=False)
    .agg({
        'Tmin': 'mean',
        'Tmin_valida': 'sum',
        'Latitud': 'first',
        'Longitud': 'first'
    })
)

# Filtrar semanas con al menos 3 días válidos
agg_tmin = agg_tmin[agg_tmin['Tmin_valida'] >= 3].drop(columns='Tmin_valida')
agg_tmin = agg_tmin.rename(columns={
    "epi_year": "year",
    "epi_week": "week"  
})
agg_tmin["epi_week"] = agg_tmin["year"].astype(str) + "-" + agg_tmin["week"].astype(str).str.zfill(2)
agg_tmin

,Estacion,year,week,Tmin,Latitud,Longitud,epi_week
4486,CAÑAVERALEJO - EDIFICIO CVC,2014,53,21.870000,3.405859,-76.538885,2014-53
4487,CAÑAVERALEJO - EDIFICIO CVC,2015,1,21.795714,3.405859,-76.538885,2015-01
4488,CAÑAVERALEJO - EDIFICIO CVC,2015,2,22.167500,3.405859,-76.538885,2015-02
4490,CAÑAVERALEJO - EDIFICIO CVC,2015,4,21.442857,3.405859,-76.538885,2015-04
4491,CAÑAVERALEJO - EDIFICIO CVC,2015,5,22.011429,3.405859,-76.538885,2015-05
...,...,...,...,...,...,...,...
16628,Pance,2023,48,19.950000,3.306903,-76.542442,2023-48
16629,Pance,2023,49,20.185714,3.306903,-76.542442,2023-49
16630,Pance,2023,50,20.585714,3.306903,-76.542442,2023-50
16631,Pance,2023,51,20.200000,3.306903,-76.542442,2023-51


## Información Satelital 

In [5]:
# Rutas a los archivos
data_gee = "D:/OneDrive - CGIAR/Desktop/downscaling/data/preprocessed/gee/gee_2013-22_2025-05.nc"
data_c3s = "D:/OneDrive - CGIAR/Desktop/downscaling/data/preprocessed/copernicus/c3s_2013-22_2025-05.nc"
data_chirps = "D:/OneDrive - CGIAR/Desktop/downscaling/data/preprocessed/chirps/chirps_2013-22_2025-05.nc"
data_dem = "D:/OneDrive - CGIAR/Desktop/downscaling/data/auxiliary/DEM/DEM.tif"


modis_crs = CRS.from_proj4(
    "+proj=sinu +R=6371007.181 +nadgrids=@null +wktext"
)

# Cargar como dataset (puede contener múltiples variables)
ds_chirps = xr.open_dataset(data_chirps).rio.write_crs("EPSG:4326", inplace=False)
ds_c3s = xr.open_dataset(data_c3s).rio.write_crs("EPSG:4326", inplace=False).rename({"lat": "y", "lon": "x"})
ds_gee = xr.open_dataset(data_gee).rio.write_crs(modis_crs)
ds_dem = xr.open_dataset(data_dem).squeeze("band").rio.write_crs("EPSG:4326", inplace=False)

# Listar variables disponibles
print("CHIRPS variables:", list(ds_chirps.data_vars))
print("C3S variables:", list(ds_c3s.data_vars))
print("GEE variables:", list(ds_gee.data_vars))
print("DEM variables:", list(ds_dem.data_vars))

CHIRPS variables: ['precip']
C3S variables: ['tmax', 'tmin', 'humidity', 'windspeed']
GEE variables: ['NDVI', 'NDWI']
DEM variables: ['band_data']


In [6]:
# Paso 1: Aplica el filtro para reemplazar por NA valores fuera del rango NDVI
NDVI_filtrado = ds_gee.NDVI.where((ds_gee.NDVI >= -1) & (ds_gee.NDVI <= 1))
# Paso 2: Crea una máscara que identifica *solo* los nuevos NaN (es decir, los que fueron filtrados)
mask_NDVI = NDVI_filtrado.isnull() & ds_gee.NDVI.notnull()
# Paso 3: Interpola
ndvi_step1 = NDVI_filtrado.interpolate_na(dim="y", method="nearest", use_coordinate=False)
NDVI_interpolado = ndvi_step1.interpolate_na(dim="x", method="nearest", use_coordinate=False)
# Paso 4: Reconstruye: deja interpolado solo donde fue filtrado; deja el resto como estaba
NDVI_500m= NDVI_filtrado.where(~mask_NDVI, NDVI_interpolado)

In [7]:
# Paso 1: Aplica el filtro para reemplazar por NA valores fuera del rango NDWI
NDWI_filtrado = ds_gee.NDWI.where((ds_gee.NDWI >= -1) & (ds_gee.NDWI <= 1))
# Paso 2: Crea una máscara que identifica *solo* los nuevos NaN (es decir, los que fueron filtrados)
mask_NDWI = NDWI_filtrado.isnull() & ds_gee.NDWI.notnull()
# Paso 3: Interpola
ndWi_step1 = NDWI_filtrado.interpolate_na(dim="y", method="nearest", use_coordinate=False)
NDWI_interpolado = ndWi_step1.interpolate_na(dim="x", method="nearest", use_coordinate=False)
# Paso 4: Reconstruye: deja interpolado solo donde fue filtrado; deja el resto como estaba
NDWI_500m = NDWI_filtrado.where(~mask_NDWI, NDWI_interpolado)

In [8]:
# Se generan variables a 1 km con resampling 
NDVI_1k = NDVI_500m.rio.reproject_match(ds_dem, resampling=Resampling.nearest)
NDWI_1k = NDWI_500m.rio.reproject_match(ds_dem, resampling=Resampling.nearest)

## Variables DEM

In [9]:
dem_da_meters = ds_dem.band_data.rio.reproject("EPSG:3116")  # MAGNA-SIRGAS / Cali urbana
tpi_da = calcular_tpi_mascarado(dem_da_meters)

# Umbrales TPI (puedes ajustar)
cumbre_mask = (tpi_da > 100).fillna(False).values
valle_mask = (tpi_da < -100).fillna(False).values

dist_cumbre = distance_transform_edt(~cumbre_mask)
dist_valle = distance_transform_edt(~valle_mask)

dist_cumbre_da = xr.DataArray(dist_cumbre, coords=dem_da_meters.coords, dims=dem_da_meters.dims, name='dist_to_cumbre')
dist_valle_da = xr.DataArray(dist_valle, coords=dem_da_meters.coords, dims=dem_da_meters.dims, name='dist_to_valle')
### Pendiente y Aspect
slope_da = slope(dem_da_meters)
aspect_da = aspect(dem_da_meters)
### Rugosity
rugosity_array = generic_filter(dem_da_meters.values, local_rugosity, size=3, mode='nearest')
rugosity_da = xr.DataArray(rugosity_array, coords=dem_da_meters.coords, dims=dem_da_meters.dims, name='rugosity')

C:\Users\dagudelo\AppData\Local\Temp\ipykernel_37356\29716890.py:83: RuntimeWarning: divide by zero encountered in divide
  mean_neighbors = np.where(count_neighbors > 0, sum_neighbors / count_neighbors, np.nan)
C:\Users\dagudelo\AppData\Local\Temp\ipykernel_37356\29716890.py:83: RuntimeWarning: invalid value encountered in divide
  mean_neighbors = np.where(count_neighbors > 0, sum_neighbors / count_neighbors, np.nan)


## Downscaling temperature saga 

In [10]:
tmax_11k = ds_c3s.tmax- 273.15
tmin_11k = ds_c3s.tmin - 273.15

tmax_saga =downscale_tmax_with_dem_from_epiweeks(tmax_11k,ds_dem,lapse_rate=-6.5)
tmin_saga = downscale_tmax_with_dem_from_epiweeks(tmin_11k,ds_dem,lapse_rate=-6.5)

## Extracción de valores para las estaciones

In [12]:
## Variables input 
DEM_1k = ds_dem.expand_dims({'epi_week': tmin_11k.epi_week})
tpi_1k = tpi_da.expand_dims({'epi_week': tmin_11k.epi_week})
cumbre_1k = dist_cumbre_da.expand_dims({'epi_week': tmin_11k.epi_week})
valle_1k = dist_valle_da.expand_dims({'epi_week': tmin_11k.epi_week})
slope_1k = slope_da.expand_dims({'epi_week': tmin_11k.epi_week})
aspect_1k = aspect_da.expand_dims({'epi_week': tmin_11k.epi_week})
rugosity_1k = rugosity_da.expand_dims({'epi_week': tmin_11k.epi_week})
NDVI_4326 = NDVI_500m.rio.reproject("EPSG:4326")

In [15]:
def extract_from_da_exact_week(row, da):
    try:
        return da.sel(
            epi_week=row["epi_week"]  # selección exacta
        ).sel(
            x=row["Longitud"],
            y=row["Latitud"],
            method="nearest"  # solo para x/y (numéricos)
        ).item()
    except Exception as e:
        print(f"Fila {row.name} → error: {e}")
        return np.nan
    
    

    
def extract_from_da_static(row, da):
    try:
        return da.sel(
            x=row["Longitud"],
            y=row["Latitud"],
            method="nearest"
        ).item()
    except Exception as e:
        print(f"Fila {row.name} → error: {e}")
        return np.nan

agg_tmin["tmax_saga"] = agg_tmin.apply(lambda row: extract_from_da_exact_week(row, tmax_saga), axis=1)
agg_tmin["tmin_saga"] = agg_tmin.apply(lambda row: extract_from_da_exact_week(row, tmin_saga), axis=1)
agg_tmin["slope"] = agg_tmin.apply(lambda row: extract_from_da_static(row, slope_da.rio.reproject("EPSG:4326")), axis=1)
agg_tmin["aspect"] = agg_tmin.apply(lambda row: extract_from_da_static(row, aspect_da.rio.reproject("EPSG:4326")), axis=1)
agg_tmin["tpi"] = agg_tmin.apply(lambda row: extract_from_da_static(row, tpi_da.rio.reproject("EPSG:4326")), axis=1)
agg_tmin["cumbre"] = agg_tmin.apply(lambda row: extract_from_da_static(row, dist_cumbre_da.rio.reproject("EPSG:4326")), axis=1)
agg_tmin["valle"] = agg_tmin.apply(lambda row: extract_from_da_static(row, dist_valle_da.rio.reproject("EPSG:4326")), axis=1)
agg_tmin["rugosity"] = agg_tmin.apply(lambda row: extract_from_da_static(row, rugosity_da.rio.reproject("EPSG:4326")), axis=1)
agg_tmin["dem"] = agg_tmin.apply(lambda row: extract_from_da_static(row, ds_dem.band_data), axis=1)
agg_tmin["ndvi"] = agg_tmin.apply(lambda row: extract_from_da_exact_week(row, NDVI_4326), axis=1)

Fila 4546 → error: "not all values found in index 'epi_week'. Try setting the `method` keyword argument (example: method='nearest')."
Fila 7674 → error: "not all values found in index 'epi_week'. Try setting the `method` keyword argument (example: method='nearest')."
Fila 9337 → error: "not all values found in index 'epi_week'. Try setting the `method` keyword argument (example: method='nearest')."
Fila 16570 → error: "not all values found in index 'epi_week'. Try setting the `method` keyword argument (example: method='nearest')."


In [16]:
agg_tmin.isna().sum()

Estacion     0
year         0
week         0
Tmin         0
Latitud      0
Longitud     0
epi_week     0
tmax_saga    0
tmin_saga    0
slope        0
aspect       0
tpi          0
cumbre       0
valle        0
rugosity     0
dem          0
ndvi         4
dtype: int64

In [17]:
agg_tmin.shape

(1418, 17)

In [18]:
agg_tmin.to_parquet("D:/OneDrive - CGIAR/Desktop/downscaling/data/features/tmin_features.parquet")  